In [36]:
import os, sys, warnings
import concurrent.futures
from openpyxl import Workbook
from multiprocessing import Pool
from pathlib import Path
from IPython.display import Image, display

warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

from selenium.webdriver.common.by import By 
from selenium.webdriver.common.keys import Keys


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
import matplotlib.pyplot as plt
from matplotlib import colors

import pandas as pd

In [38]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

In [39]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install().replace("THIRD_PARTY_NOTICES.", "")))

In [40]:
# Def the website i want to extract the table for
url = 'https://www.coingecko.com/pt'
driver.get(url)

In [41]:
# Picking table from Crypto Coins site and creating a variable for the rows in this table
table = driver.find_element(By.XPATH, '/html/body/div[2]/main/div/div[5]/table')
rows = table.find_elements(By.XPATH, './/tbody//tr')

In [42]:
def maybe_float(s):
    try:
        return round(float(s.replace(".", "").replace(",", ".")), 2)
    except (ValueError, TypeError):
        return s

In [43]:
# Creating the function to iterate in rows and cells of the table and appending to the table_data
def process_cells(cells):
    cell_data = []
    for cell in cells:
        img_tag = cell.find_elements(By.TAG_NAME, 'img')
        data_sort = cell.get_attribute('data-sort')
        if img_tag:
            # Extract the image URL (src attribute)
            img_src = img_tag[0].get_attribute('src')
            cell_data.append(img_src)

        if '$' in cell.text:
            cell_splited = cell.text.split(" ")
            for part_splitted in cell_splited:  
                cell_data.append(maybe_float(part_splitted))

        elif data_sort:
            cell_data.append(round(float(data_sort), 2))
            
        else:
            cell_data.append(cell.text)
    return cell_data

In [44]:
def process_rows(rows):
    table_data = []
    for row in rows:
        cells = row.find_elements(By.XPATH, './/td')
        cells_data = process_cells(cells)
        table_data.append(cells_data)
    return table_data

In [45]:
table_data = process_rows(rows)

In [46]:
headers = [header.text for header in table.find_elements(By.XPATH, './/thead//th')]
headers = [item for item in headers if item]

headers[0] = "IMG Coin"
insertions = [(0, "ID"), (3, "$"), (8, "30 d"), (9, "$"), (11, "$"), (13, "FDV")
]
for index, value in insertions:
    headers.insert(index, value)

In [ ]:
df = pd.DataFrame(table_data)
df_treated_with_link = df.loc[:, (df != '').any(axis=0)] #test drop_na
df_treated_with_link = df_treated_with_link.drop(df.columns[4], axis=1)
df_treated_with_link = df_treated_with_link.reset_index(drop=True)
df_treated_with_link


In [ ]:
df_treated_with_link.style\
    .set_properties(**{'border': '1.3px solid black', 'color': 'white'})\
    .background_gradient(subset=[6])
    #


In [49]:
df_treated_with_link.columns = [headers]
#df_treated_with_link

In [50]:
df_treated = df_treated_with_link.drop(columns=['IMG Coin', 'Últimos 7 dias'])

In [ ]:
df_treated

In [52]:
df_treated.to_excel("file_test_df_cripto_coin.xlsx", index=False)